In [8]:
from pathlib import Path

import pandas as pd

import insightify.config as config
import insightify.libs.utils_io as utils_io

In [15]:
config.dump()

path_dir_data = Path(config._PATH_DIR_DATA)

{'_logger': <Logger insightify.config (WARNING)>,
 '_path_config_package': '/home/bertan/repos/insightify/insightify/config.yaml',
 '_path_dir_data': '/home/bertan/repos/insightify/data',
 '_seed_rngs': 42}


In [ ]:
path_marquee = path_dir_data / "Spotify Account Data" / "Marquee.json"
df_marquee = pd.DataFrame(utils_io.read_json(path_marquee))

display(df_marquee.tail())

for segment, group in df_marquee.groupby("segment"):
    print(f"\nSegment: {segment}")
    print(group)

,artistName,segment
544,Star Parks,Light listeners
545,The Mowgli's,Light listeners
546,Rise Against,Previously Active Listeners
547,Imagine Dragons,Moderate listeners
548,Bishop Briggs,Previously Active Listeners



Segment: Light listeners
            artistName          segment
1               HUNGER  Light listeners
2       Gizmo Varillas  Light listeners
4              Son Lux  Light listeners
5          Giant Rooks  Light listeners
6         Cemetery Sun  Light listeners
..                 ...              ...
534  Father John Misty  Light listeners
535     The Envy Corps  Light listeners
541      Guns N' Roses  Light listeners
544         Star Parks  Light listeners
545       The Mowgli's  Light listeners

[90 rows x 2 columns]

Segment: Moderate listeners
          artistName             segment
28      Agent Fresco  Moderate listeners
50          Daughter  Moderate listeners
82        Blacksmith  Moderate listeners
245       Dan Mangan  Moderate listeners
334             OWEL  Moderate listeners
354      A. Sinclair  Moderate listeners
387   Blonde Redhead  Moderate listeners
405            Meltt  Moderate listeners
437              Mew  Moderate listeners
480             IAMX  Moderate l

In [32]:
path_library = path_dir_data / "Spotify Account Data" / "YourLibrary.json"
df_library_artists = pd.DataFrame(utils_io.read_json(path_library)["artists"])
df_library_artists = df_library_artists.drop("uri", axis=1)

display(df_library_artists.tail())

,name
212,flor
213,flora cash
214,isaac gracie
215,nervous_testpilot
216,the sleepover disaster


In [41]:
path_library = path_dir_data / "Spotify Account Data" / "YourLibrary.json"
df_library_tracks = pd.DataFrame(utils_io.read_json(path_library)["tracks"])
df_library_tracks = df_library_tracks.drop("uri", axis=1)

display(df_library_tracks.tail())
display(df_library_tracks.groupby("artist").count()["track"].sort_values(ascending=False).head(10))

,artist,album,track
2142,Mew,+ -,Making Friends
2143,Antilopen Gang,Anarchie und Alltag + Bonusalbum Atombombe auf...,Pizza
2144,Broken Bells,After the Disco,Perfect World
2145,Amarante,Udana Prana,The Manic
2146,Wolf Gang,Suego Faults,Back to Back


artist
Imagine Dragons             56
Antilopen Gang              45
The Airborne Toxic Event    41
Von Wegen Lisbeth           36
A. Sinclair                 35
Mew                         32
Blonde Redhead              32
Polock                      31
Agent Fresco                30
Everything Everything       27
Name: track, dtype: int64

In [226]:
path_dir_history = path_dir_data / "Spotify Extended Streaming History"

df_history = pd.DataFrame()
for path_history in sorted(path_dir_history.glob("*json")):
    df_history_partial = pd.DataFrame(utils_io.read_json(path_history))
    df_history = pd.concat((df_history, df_history_partial), ignore_index=True)

df_history = df_history.drop(["platform", "conn_country", "spotify_track_uri", "episode_name", "episode_show_name", "spotify_episode_uri"], axis=1)
df_history["year"] = pd.to_datetime(df_history["ts"]).dt.year
display(df_history.head())

,ts,username,ms_played,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,year
0,2015-06-11T15:04:48Z,bertööö,12120,84.135.195.94,unknown,Where Are Ü Now (with Justin Bieber),Jack Ü,Skrillex and Diplo present Jack Ü,trackdone,fwdbtn,False,True,False,0.0,False,2015
1,2015-06-11T15:05:07Z,bertööö,19411,84.135.195.94,unknown,King,Olly Alexander (Years & Years),King,fwdbtn,fwdbtn,False,True,False,0.0,False,2015
2,2015-06-11T15:05:10Z,bertööö,4133,84.135.195.94,unknown,Traffic Lights,Lena,Crystal Sky,fwdbtn,fwdbtn,False,True,False,0.0,False,2015
3,2015-06-11T15:05:19Z,bertööö,9705,84.135.195.94,unknown,Want to Want Me,Jason Derulo,Everything Is 4,fwdbtn,fwdbtn,False,True,False,0.0,False,2015
4,2015-06-11T15:05:27Z,bertööö,9195,84.135.195.94,unknown,Thinking out Loud,Ed Sheeran,x,fwdbtn,fwdbtn,False,True,False,0.0,False,2015


In [214]:
print(df_history["reason_end"].unique())
df_history[df_history["reason_end"].isin(("fwdbtn", "backbtn"))].count()

['fwdbtn' 'endplay' 'trackdone' 'unknown' 'logout' 'unexpected-exit'
 'unexpected-exit-while-paused' 'backbtn' None 'remote' 'trackerror']


ts                                   7360
username                             7360
ms_played                            7360
ip_addr_decrypted                    7360
user_agent_decrypted                 7340
master_metadata_track_name           7313
master_metadata_album_artist_name    7313
master_metadata_album_album_name     7313
reason_start                         7360
reason_end                           7360
shuffle                              7360
skipped                              1018
offline                              7360
offline_timestamp                    7360
incognito_mode                       7360
dtype: int64

In [218]:
df_history.loc[:, "skipped"] = df_history["reason_end"].isin(("fwdbtn",))

df_history_filtered = df_history[~df_history["skipped"].isnull()]
df_history_filtered = df_history_filtered[df_history_filtered["master_metadata_track_name"].isin(df_library_tracks["track"])]
df_history_filtered.loc[:, "skipped"] = df_history_filtered["skipped"].astype(bool)

counts = df_history_filtered.groupby(["master_metadata_track_name", "skipped"]).size().unstack(fill_value=0)
counts["total"] = counts[True] + counts[False]
counts["true_ratio"] = counts[True] / counts["total"]
sorted_counts = counts.sort_values(by="true_ratio", ascending=False)
print(sorted_counts[sorted_counts["total"] >= 5].to_string())

# df_history_filtered[df_history_filtered["skipped"]].groupby("master_metadata_album_artist_name")

# df_history["reason_end"]

skipped                                                                                         False  True  total  true_ratio
master_metadata_track_name                                                                                                    
Forgiveness I + II                                                                                  2     7      9    0.777778
Walk                                                                                               11    22     33    0.666667
How Soon Is Now? - 2011 Remaster                                                                    2     4      6    0.666667
Coyote                                                                                              3     5      8    0.625000
Once in a Long, Long While...                                                                       2     3      5    0.600000
Half The World Away                                                                                 6     9    

In [229]:
df_history[df_history["year"] > 2019].groupby(["master_metadata_album_artist_name"])["ms_played"].sum().sort_values(ascending=False).head(20) / (1000 * 60 * 60)

master_metadata_album_artist_name
Imagine Dragons             22.841529
The Airborne Toxic Event    22.749691
Polock                      21.386071
Mew                         20.392302
Blonde Redhead              18.433679
A. Sinclair                 15.700947
Everything Everything       13.021809
Faunts                      12.207431
Au4                         12.049897
Racing Glaciers             11.971668
Arcade Fire                 11.501316
Radiohead                   11.460452
The xx                      10.002502
Foals                        9.643972
IAMX                         9.598296
The Neighbourhood            9.534020
Father John Misty            9.529219
Sunset Sons                  9.239225
Son Lux                      9.042116
The Notwist                  8.990638
Name: ms_played, dtype: float64